In [1]:
import pandas as pd
import numpy as np

In [2]:

PATH_NWIPR = 'D:\\Run Analysis\\BLOB_STORAGE\\Visy_Contract_Report_NIWPR\\Apr_2021.xlsx'
PATH_ROS = 'D:\\Run Analysis\\BLOB_STORAGE\\Roster\\weekly_roster_processed\\14th_2021.xlsx'
PATH_FUEL_A = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\fuel\\Caltex_fuel_transactions\\mar_apr_2021.html'
PATH_FUEL_B = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\fuel\\Caltex_fuel_transactions\\apr_may_2021.html'

# pandas read
# =========================================
df_visy = pd.read_excel(PATH_NWIPR)
df_ros = pd.read_excel(PATH_ROS)
df_fuel_a = pd.read_html(PATH_FUEL_A, converters={'Card Number' : np.int64 })
df_fuel_b = pd.read_html(PATH_FUEL_B,converters={'Card Number' : np.int64 })

In [3]:
# Date
df_fuel_a[0]['Trans. Date'] = pd.to_datetime(df_fuel_a[0]['Trans. Date'], format='%d/%m/%Y')
df_fuel_a = df_fuel_a[0].rename(columns={'Trans. Date' : 'Date'})

df_fuel_b[0]['Trans. Date'] = pd.to_datetime(df_fuel_b[0]['Trans. Date'], format='%d/%m/%Y')
df_fuel_b = df_fuel_b[0].rename(columns={'Trans. Date' : 'Date'})

In [4]:
df_a = df_fuel_a[df_fuel_a.Date.between('20210407', '20210413')]
df_b = df_fuel_b[df_fuel_b.Date.between('20210407', '20210413')]

df_fuel_14th_2021 = pd.concat([df_a, df_b])

In [5]:
df_justice = df_fuel_14th_2021[df_fuel_14th_2021['Card Number'].eq(7071340093371952)]
df_justice['Amount Ex GST'] = df_justice['Amount Ex GST'].str.replace('$', '').astype('float64')

# Week 14th Justice Fuel cost  
df_justice['Amount Ex GST'].sum()
# df_fuel_14th_2021[df_fuel_14th_2021['Card Number'].eq(7071340079630116)]


<ipython-input-5-0e712709fa64>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_justice['Amount Ex GST'] = df_justice['Amount Ex GST'].str.replace('$', '').astype('float64')


348.17999999999995

In [7]:
COMPLETE_PATH = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\fuel\\NIWPR\\14th_2021_NIWPRFUEL.xlsx'
df_justice.to_excel(COMPLETE_PATH)

In [24]:
PATH_ROS = 'D:\\Run Analysis\\BLOB_STORAGE\\Roster\\weekly_roster_processed\\14th_2021.xlsx'
df_ros = pd.read_excel(PATH_ROS)
df_ros.Primary_truck = df_ros.Primary_truck.str.replace(' ','')


PATH_REGO = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\Rego_2021.xlsx'
df_rego = pd.read_excel(PATH_REGO)
df_rego = df_rego[['Rego', 'weekly_rego_payment']]

In [25]:
occur = df_ros.groupby(['Primary_truck', 'Primary_route'])['Run_type'].count().reset_index()
total = df_ros.groupby('Primary_truck')['Primary_route'].count().reset_index()

df_counts  = pd.merge(occur, total, how='left', on='Primary_truck')

df_counts['portion'] = df_counts['Run_type'] / df_counts['Primary_route_y']
# df_counts 

df_rego_counts = pd.merge(df_counts, df_rego, how='left', left_on='Primary_truck', right_on='Rego')

In [27]:
df_rego_counts['amount'] = df_rego_counts.weekly_rego_payment * df_rego_counts.portion

In [30]:
df_rego_counts.groupby('Primary_route_x')['amount'].sum().reset_index()

,Primary_route_x,amount
0,BR1,73.528462
1,BR2,73.528462
2,BR3,78.520128
3,CAMP,0.000000
4,DOY,10.019103
5,FL2,73.316731
6,FLG,57.089423
7,HOOK1,59.234423
8,HYG,37.192308
9,NIWPR1,0.000000


In [12]:
PATH_REGO = 'D:\\Run Analysis\\BLOB_STORAGE\\expenses_truck\\Rego_2021.xlsx'
df_rego = pd.read_excel(PATH_REGO)
# df_rego